In [ ]:
# NOTE!! This version is with raw byonic excel file
# NOTE!! Here I assume byonic does not contain repeated scans
# NOTE!! has id but target not present -> 'N/A', no id at all -> -1 (align data func default) 
# file: pGlycoDB-GP-FDR-Pro-Quant-Site
# PepScore>5,GlyScore>4
# compare the results to that of byonic (score > 200, pep2d < 0.001)
# This is a generalized version for file w/ only hcd or both
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib import ticker
from IPython.display import display, HTML
import re # finding specific patterns in str
import textwrap # split text into equal parts
import collections # return repeated items in list
from collections import OrderedDict
import time

start_time = time.time()

# read in the files as df
pglycofile = '20210521_NL63_human_ingelTC_pGlycoDB-GP-FDR-Pro-Quant-Site'
pglyco_df = pd.read_excel('%s.xlsx'%pglycofile, header = 0)
pglyco_df = pglyco_df.fillna('N/A')
pglyco_df = pglyco_df.sort_values(by=['Scan'])
# display(HTML(df.to_html()))

## preprocess the pglycofile first

# output column name
print('original pglyco columns:\n%s\n'%pglyco_df.columns)

# record original data size
print('original data size:\nrow: %s\ncol: %s\n'%(pglyco_df.shape[0], pglyco_df.shape[1]))

# change J back to N as column named peptide(J-->N)
pglyco_df['Peptide'] = pglyco_df['Peptide'].str.replace('J','N')

# analyze sequon in pglyco file
pglyco_sequon = pglyco_df['Peptide'].str.findall('(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)').tolist()
pglyco_sequon_lst = []
for t in pglyco_sequon:
    t = str(t)
    res = re.findall('[ARNDBCEQZGHILKMFSTWYVP]', t)
    res = ''.join(res)
    res = textwrap.wrap(res, 3)
    if res == []:
        pglyco_sequon_lst.append('N/A')
    elif len(res) == 1:
        pglyco_sequon_lst.append(res[0])
    else:
        pglyco_sequon_lst.append(res)
        
# print('pglyco_sequon: %s'%pglyco_sequon_lst)
pglyco_df.insert(pglyco_df.columns.get_loc('Peptide') + 1 , 'Sequon', pglyco_sequon_lst , True)
# display(HTML(pglyco_df.to_html()))

# replace H, N, A, F symbols w/ HexNAc, Hex, NeuAc, Fuc
GlycanComposition = []
for i in pglyco_df['GlycanComposition'].tolist():
#     print('i:%s'%i)
    i = i.split(')')
#     print('split: %s'%i)
    if any('H' in x for x in i) and any('N' in x for x in i) and len(i) > 3:
        if any('A' in x for x in i) and any('F' in x for x in i): # HNAF all present
#             print(i)
            order = [1, 0, 3, 2, 4]
            i = [i[p] for p in order]
            i = ')'.join(i)
            GlycanComposition.append(i)
#             print('reordered: %s'%i)
        else: # HNA or HNF
#             print(i)
            order = [1, 0, 2, 3]
            i = [i[p] for p in order]
            i = ')'.join(i)
            GlycanComposition.append(i)
#             print('reordered: %s'%i)
    elif any('H' in x for x in i) and any('N' in x for x in i) and len(i) == 3:
#         print(i)
        order = [1, 0, 2]
        i = [i[p] for p in order]
        i = ')'.join(i)
        GlycanComposition.append(i)
#         print('reordered: %s'%i)
    elif any('H' in x for x in i) and any('N' in x for x in i) == False: # only has N
        i = ')'.join(i)
        GlycanComposition.append(i)
#         print('reordered: %s'%i)
# print(GlycanComposition)    
# print(len(GlycanComposition))
pglyco_df['GlycanComposition'] = GlycanComposition
p, b = ['H','N','A', 'F'], ['Hex','HexNAc','NeuAc', 'Fuc']
rep = dict(zip(p, b))
GlycanComposition_ByonicStyle = pglyco_df['GlycanComposition'].replace(rep, regex = True)

## replace 'HexNNeuAcc' w/ HexNAc'
rep = {'HexNNeuAcc': 'HexNAc'}
GlycanComposition_ByonicStyle = GlycanComposition_ByonicStyle.replace(rep, regex = True)
pglyco_df.insert(pglyco_df.columns.get_loc('GlycanComposition') + 1 , 'GlycanComposition_ByonicStyle', GlycanComposition_ByonicStyle , True)
# display(HTML(pglyco_df.to_html()))

## import byonic raw excel file (contain all the info.) to compare
byonicfile = '20201124_in_house_NL63_S_2P_NGlycan'
byonic_df = pd.read_excel('%s.xlsx'%byonicfile, header = 0)
byonic_df = byonic_df.fillna('N/A')
# output column name
print('original byonic columns:\n%s\n'%byonic_df.columns)
# display(HTML(byonic_df.to_html()))

# extract 'scan' from 'scan #' in byonic file & add a 'Scan' column
byonic_scan = byonic_df['Scan #'].tolist()
byonic_scan_lst = []
for scan in byonic_scan:
    scan = scan.split(' ')[-1].split('=')[-1]
    scan = int(scan)
#     print(scan)
    byonic_scan_lst.append(scan)
# print(byonic_scan_lst)
byonic_df.insert(byonic_df.columns.get_loc('Scan\r\nTime') + 1 , 'Scan', byonic_scan_lst , True)
byonic_df = byonic_df.sort_values(by=['Scan'])

# add 'PureSequence' column to byonic file
byonic_seq = byonic_df['Sequence\r\n(unformatted)'].str.findall('[ARNDBCEQZGHILKMFSTWYVP]').tolist()
byonic_seq = [''.join(each_pure) for each_pure in byonic_seq]
byonic_df.insert(byonic_df.columns.get_loc('Sequence\r\n(unformatted)') + 1 , 'PureSequence', byonic_seq , True)

# add 'Sequon' column to byonic file
byonic_sequon_lst = []
byonic_sequon = byonic_df['PureSequence'].str.findall('(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)').tolist()
for t in byonic_sequon:
    t = str(t)
    res = re.findall('[ARNDBCEQZGHILKMFSTWYVP]', t)
    res = ''.join(res)
    res = textwrap.wrap(res, 3)
    if res == []:
        byonic_sequon_lst.append('N/A')
    elif len(res) == 1:
        byonic_sequon_lst.append(res[0])
    else:
        byonic_sequon_lst.append(res)
byonic_df.insert(byonic_df.columns.get_loc('PureSequence') + 1 , 'Sequon', byonic_sequon_lst , True)
# display(HTML(byonic_df.to_html()))
    
# extract scan from pglyco file
pglyco_scan = pglyco_df['Scan'].tolist()

## combine scans from the two files

# check for duplicates or more in byonic & pglyco scan
sorted_byonic_scan = byonic_df['Scan'].tolist()
byonic_repeated_scan = [item for item, count in collections.Counter(sorted_byonic_scan).items() if count > 1]
pglyco_repeated_scan = [item for item, count in collections.Counter(pglyco_scan).items() if count > 1]
byonic_repeated_count = [count for item, count in collections.Counter(sorted_byonic_scan).items() if count > 1]
pglyco_repeated_count = [count for item, count in collections.Counter(pglyco_scan).items() if count > 1]
byonic_repeated_dict = dict(zip(byonic_repeated_scan, byonic_repeated_count))
pglyco_repeated_dict = dict(zip(pglyco_repeated_scan, pglyco_repeated_count))
print('repeated scans in byonic file {scan:times}: %s'%(byonic_repeated_dict))
print('repeated scans in pglyco file {scan:times}: %s'%(pglyco_repeated_dict))

# if the repeated scan from two files overlaps, then only choose the one w/ more times & add corresponding rows
combined_scan = byonic_scan_lst.copy()
combined_scan.extend(pglyco_scan)
combined_scan = sorted(set(combined_scan))
keys = set(byonic_repeated_dict.keys()).union(pglyco_repeated_dict.keys())
repeat = {k:max(byonic_repeated_dict.get(k,float('-inf')), pglyco_repeated_dict.get(k, float('-inf'))) for k in keys}
for k in repeat.keys():
    for i in range(repeat[k] - 1):
        combined_scan.append(k)
combined_scan = sorted(combined_scan)


## start combining data

# # create all the required column lists (w/ blank item to fit the length of scan)
# sorted_byonic_scan = byonic_df['Scan'].tolist()

# isolate byonic df (scan, sequon, glycan, pep2d, score) & pglyco (whole dt), then set scan as index for both
# byonic_scanasind = byonic_df[['Scan', 'ByonicSequon', 'Glycans', 'PEP\r\n2D', 'Score']]
byonic_scanasind = byonic_df.copy()
new_byonic_col = [n + '[Byonic]' if n != 'Scan' else n for n in byonic_scanasind.columns]
# print('new_byonic_col: %s'%new_byonic_col)
byonic_scanasind.columns = new_byonic_col
pglyco_scanasid = pglyco_df.copy()
new_pglyco_col = [n + '[pGlyco]' if n != 'Scan' else n for n in pglyco_scanasid.columns]
# print(new_pglyco_col)
pglyco_scanasid.columns = new_pglyco_col
byonic_scanasind = byonic_scanasind.set_index('Scan')
pglyco_scanasid = pglyco_scanasid.set_index('Scan')

# align the scan & concat
a1, a2 = byonic_scanasind.align(pglyco_scanasid, join = 'outer', axis = 0)
all_combined_df = pd.concat([a1,a2], axis = 1)
all_combined_df.reset_index(level=0, inplace=True)

# set the redundant rows (due to df alignment) in byonic section to nan
nan_ind = []
for scan in repeat:
#     print(scan)
    each_nan_ind = all_combined_df[all_combined_df['Scan'] == scan].index.tolist()[1:]
    nan_ind.append(each_nan_ind)
# print(nan_ind)
for ind in nan_ind:
    all_combined_df.iloc[ind, 1:37] = -1
    
# change all nan to blank -1
all_combined_df = all_combined_df.fillna(-1)

## result post-processing
# glycan comprison: only present in byonic -> b, only present in pglyco -> p, both the same -> b+p, not the same -> b/p
conditions = [
    (all_combined_df['Glycans[Byonic]'] != -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] == -1),
    (all_combined_df['Glycans[Byonic]'] == -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] != -1),
    (all_combined_df['Glycans[Byonic]'] != -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (all_combined_df['Glycans[Byonic]'] == all_combined_df['GlycanComposition_ByonicStyle[pGlyco]']),
    (all_combined_df['Glycans[Byonic]'] != -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (all_combined_df['Glycans[Byonic]'] != all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'])]
choices = ['B', 'P', 'B+P', 'B/P'] 
glycan_source = np.select(conditions, choices) 
all_combined_df.insert(all_combined_df.columns.get_loc('Scan') + 1 , 'GlycanSource', glycan_source , True)

print('combined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))

## style apply for excel export
# color the rows below the threshold (threshold [byonic: score > 200 & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])

# color the background to separate byonic data from pglyco data
# color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco

def bg_color(x):
    c1 = 'background-color: #98FB98' 
    c2 = 'background-color: #add8e6'
    c3 = 'background-color: #ffedcc'
    c = '' 
    #compare columns
    b_mask = (all_combined_df['Score[Byonic]'] > 200) & (all_combined_df['PEP\r\n2D[Byonic]'].abs() < 0.001) & (all_combined_df['PepScore[pGlyco]'] <= 5) & (all_combined_df['GlyScore[pGlyco]'] <= 4)
    p_mask = (all_combined_df['Score[Byonic]'] <= 200) & (all_combined_df['PEP\r\n2D[Byonic]'].abs() >= 0.001) & (all_combined_df['PepScore[pGlyco]'] > 5) & (all_combined_df['GlyScore[pGlyco]'] > 4)    
    both_mask = (all_combined_df['Score[Byonic]'] > 200) & (all_combined_df['PEP\r\n2D[Byonic]'].abs() < 0.001) & (all_combined_df['PepScore[pGlyco]'] > 5) & (all_combined_df['GlyScore[pGlyco]'] > 4)
    #DataFrame with same index and columns names as original filled empty strings
    df1 =  pd.DataFrame(c, index=x.index, columns=x.columns)
    #modify values of df1 column by boolean mask
    df1[b_mask] = c1
    df1[p_mask] = c2
    df1[both_mask] = c3
    return df1

all_combined_df.style.apply(bg_color, axis=None).to_excel('byonic_pglyco_combined.xlsx', index = False)  
print("execution time: --- %s seconds ---" % (time.time() - start_time))